<a href="https://colab.research.google.com/github/Sanket0412/Embedded-Text-Detection-in-Video/blob/main/Embedded_Text_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract 
! apt install tesseract-ocr
! apt install libtesseract-dev

  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=cb15be68cb19d44a41b16b4acd17d8fce2e5f6880285ee8691db86342d9ab0a7
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,58

In [107]:
import cv2
import numpy as np
import pytesseract
from google.colab.patches import cv2_imshow

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

In [77]:
def find_contours(dilated_img):
  cnts = cv2.findContours(dilated_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      ar = w / float(h)
      if ar < 5:
          cv2.drawContours(dilated_img, [c], -1, (0,0,0), -1)
  return dilated_img

In [110]:
def text_detection1(img):
    orig = img
    image = img

    gray = get_grayscale(image)
    #img = opening(img)
    img = thresholding(gray)
    #img = canny(gray)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 0, 218])
    upper = np.array([157, 54, 255])
    mask = cv2.inRange(hsv, lower, upper)

    # Create horizontal kernel and dilate to connect text characters
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,3))
    dilate1 = cv2.dilate(mask, kernel, iterations=10)
    dilate2 = cv2.dilate(img, kernel, iterations=10)

    # Find contours and filter using aspect ratio
    # Remove non-text contours by filling in the contour
    dilate1 = find_contours(dilate1)

    #dilate2 = find_contours(dilate2)
    # Detecting text on a masked image
    data1 = pytesseract.image_to_string(mask, lang='eng',config='--psm 6')
    
    # Bitwise dilation and Inversion of the image
    result2 = 255 - cv2.bitwise_and(dilate1, mask)

    #Detecting text on the dilated and inversed image
    data2 = pytesseract.image_to_string(result2, lang='eng',config='--psm 6')

    # Bitwise dilation and Inversion of the image
    result3 = img

    data3 = pytesseract.image_to_string(result3, lang='eng',config='--psm6')
    #cv2_imshow(mask)
    #cv2_imshow(dilate)
    #cv2_imshow(result1)
    cv2_imshow(mask)
    print('Text detected on the masked image:')
    print(data1)
    cv2_imshow(result2)
    print('Text detected on the masked image after dilation and inversion:')
    print(data2)
    cv2_imshow(result3)
    print('Text detected on the opening image after dilation and inversion:')
    print(data3)
    cv2.waitKey()

## Text detection on a video

In [ ]:
import numpy as np
import cv2
cap = cv2.VideoCapture('/content/drive/MyDrive/OffNoteLabsTask/trimmedRickRoll.mp4')
#cap.set(cv2.CAP_PROP_FPS, 10000) 
i = 0;
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    i+=1
    text_detection1(frame)
    #cv2_imshow(frame)
    #cv2.imwrite('RickRoll'+str(i)+'.jpg',frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
print(i)
cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2
cap = cv2.VideoCapture('/content/drive/MyDrive/OffNoteLabsTask/trimmedNews.mp4')
#cap.set(cv2.CAP_PROP_FPS, 10000) 
i = 0;
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    i+=1
    text_detection1(frame)
    #cv2_imshow(frame)
    #cv2.imwrite('RickRoll'+str(i)+'.jpg',frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
print(i)
cap.release()
cv2.destroyAllWindows()

## Text Detection on an image

In [ ]:
img = cv2.imread('/content/drive/MyDrive/OffNoteLabsTask/img.jpg')
cv2_imshow(img)
text_detection1(img)

In [ ]:
img = cv2.imread('/content/drive/MyDrive/OffNoteLabsTask/news.JPG')
cv2_imshow(img)
text_detection1(img)

In [ ]:
img = cv2.imread('/content/drive/MyDrive/OffNoteLabsTask/sampleText.png')
cv2_imshow(img)
text_detection1(img)

In [ ]:
img = cv2.imread('/content/drive/MyDrive/OffNoteLabsTask/sampleText3.jpg')
cv2_imshow(img)
text_detection1(img)

In [ ]:
img = cv2.imread('/content/drive/MyDrive/OffNoteLabsTask/sampleText4.jpg')
cv2_imshow(img)
text_detection1(img)

In [ ]:
img = cv2.imread('/content/drive/MyDrive/OffNoteLabsTask/words.jpg')
cv2_imshow(img)
text_detection1(img)